In [25]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
from sklearn.metrics import classification_report,confusion_matrix

# Path to image folders, 
data_path = fr'../Datasets/config'

def remove_white_background(pixels):
    newPixels = []
    for pixel in pixels:
        pixel = list(pixel)
        if ((256 > pixel[0] > 200) and (256 > pixel[1] > 200) and (256 > pixel[2] > 200)):
            pixel[0] = 0
            pixel[1] = 0
            pixel[2] = 0
        newPixels.append(pixel)
    
    return newPixels


def redify(pixels):
    return [r for r, g, b in pixels]

                
def greenify(pixels):
    return [g for r, g, b in pixels]


def blueify(pixels):
    return [b for r, g, b in pixels]


def get_rgb_pixels_onehot_labels(src):
    print("Starting...")
    newPixels = []
    y = np.empty(shape=[0, 1])

    for subdir in os.listdir(src):
        current_path = os.path.join(src, subdir)
        for file in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, file))
            imgResize = img.resize((24,24))
            pixels = list(imgResize.getdata())
            pixels = remove_white_background(pixels)
            newPixels.append(pixels)
            y = np.append(y, subdir)
    return newPixels, LabelBinarizer().fit_transform(y) # OneHot encode y


def process_files(src):
    X_red_train = []
    X_red_validation = []
    X_red_test = []
    X_green_train = []
    X_green_validation = []
    X_green_test = []
    X_blue_train = []
    X_blue_validation = []
    X_blue_test = []
    all_pixels, y = get_rgb_pixels_onehot_labels(src)

    X_train, X_validation, y_train, y_validation = train_test_split(all_pixels, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)
    
    for pixels in X_train:       
        X_red_train.append(redify(pixels.copy()))
        X_green_train.append(greenify(pixels.copy()))
        X_blue_train.append(blueify(pixels.copy()))
        
    for pixels in X_validation:       
        X_red_validation.append(redify(pixels.copy()))
        X_green_validation.append(greenify(pixels.copy()))
        X_blue_validation.append(blueify(pixels.copy()))
        
    for pixels in X_test:       
        X_red_test.append(redify(pixels.copy()))
        X_green_test.append(greenify(pixels.copy()))
        X_blue_test.append(blueify(pixels.copy()))
    
    
    print("Finished \n")
    return np.asarray(X_red_train), np.asarray(X_red_validation), np.asarray(X_red_test), np.asarray(X_green_train), np.asarray(X_green_validation), np.asarray(X_green_test), np.asarray(X_blue_train), np.asarray(X_blue_validation), np.asarray(X_blue_test), y_train, y_validation, y_test


def get_youdens_index(predictions, Y):
    # Calculate true positive/negative and false positive/negative
    tp = sum((Y == predictions) * (Y == 1) * 1)
    tn = sum((Y == predictions) * (Y == 0) * 1)
    fp = sum((Y != predictions) * (Y == 0) * 1)
    fn = sum((Y != predictions) * (Y == 1) * 1)
    
    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (fp + tn)
    
    result = sensitivity - (1 - specificity)
    # Put it in a dateframe for nicer visuals
    df = pd.DataFrame({'Youdens Index': result})
    pd.set_option('display.max_rows', 200)
    
    return df

In [26]:
import time

# Call process_files and assign variables
X_red_train, X_red_validation, X_red_test, X_green_train, X_green_validation, X_green_test, X_blue_train, X_blue_validation, X_blue_test, Y_train, Y_validation, Y_test = process_files(data_path)
# Fit/train train-datasets and store prediction vectors in variables
from sklearn import svm
from sklearn.neural_network import MLPClassifier

# Red Config HL Test
print("Red Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl)
    t0 = time.time()
    mlp.fit(X_red_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_red_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_red_validation), Y_validation), "\n")

# Green Config HL Test
print("Green Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl)
    t0 = time.time()
    mlp.fit(X_green_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_green_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_green_validation), Y_validation), "\n")
    
# Blue Config HL Test
print("Blue Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl)
    t0 = time.time()
    mlp.fit(X_blue_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_blue_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_blue_validation), Y_validation), "\n")

Starting...
Finished 

Red Config Tests
Hidden layers:  432
Score:  0.9862542955326461 Time:  2.572378396987915
   Youdens Index
0       1.000000
1       0.977109
2       0.989899
3       1.000000
4       1.000000
5       0.990805 

Hidden layers:  216
Score:  0.9707903780068728 Time:  1.0659687519073486
   Youdens Index
0       1.000000
1       0.968750
2       0.987829
3       0.941276
4       0.995918
5       0.990805 

Hidden layers:  (216, 108)
Score:  0.9810996563573883 Time:  1.4192888736724854
   Youdens Index
0       0.998028
1       0.979167
2       0.979798
3       0.986301
4       1.000000
5       0.986207 

Hidden layers:  108
Score:  0.9570446735395189 Time:  0.552501916885376
   Youdens Index
0       0.980750
1       0.958462
2       0.965556
3       0.994106
4       0.997959
5       0.981703 

Hidden layers:  (108, 54)
Score:  0.9621993127147767 Time:  0.8077337741851807
   Youdens Index
0       0.889389
1       0.989583
2       0.985758
3       0.994106
4       1.00000

In [32]:
# default alpha=0.0001

print("Red Config Alpha Tests")
# Red Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_red_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_red_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_red_validation), Y_validation), "\n")
    
print("Green Config Alpha Tests")
# Green Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_green_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_green_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_green_validation), Y_validation), "\n")
    
print("Blue Config Alpha Tests")
# Blue Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_blue_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_blue_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_blue_validation), Y_validation), "\n")    

Red Config Alpha Tests
Alpha:  0.0001
Score:  0.9604810996563574 Time:  0.788548469543457
   Youdens Index
0       0.960000
1       0.941744
2       0.947425
3       0.986301
4       1.000000
5       0.981609 

Alpha:  0.001
Score:  0.9690721649484536 Time:  0.6332933902740479
   Youdens Index
0       0.944694
1       0.962577
2       0.977728
3       0.986301
4       1.000000
5       0.986301 

Alpha:  0.01
Score:  0.9673539518900344 Time:  0.9388542175292969
   Youdens Index
0       0.942722
1       0.952160
2       0.987829
3       1.000000
4       1.000000
5       0.986301 

Alpha:  0.1
Score:  0.9570446735395189 Time:  0.9438574314117432
   Youdens Index
0       0.944694
1       0.968750
2       0.951314
3       0.968673
4       1.000000
5       0.970396 

Alpha:  1


c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Score:  0.9621993127147767 Time:  3.197904348373413
   Youdens Index
0       0.982722
1       0.975051
2       0.987578
3       0.986301
4       1.000000
5       0.972601 

Green Config Alpha Tests
Alpha:  0.0001
Score:  0.9759450171821306 Time:  0.3292996883392334
   Youdens Index
0       0.940750
1       0.989583
2       0.987829
3       0.982372
4       0.997959
5       0.990898 

Alpha:  0.001
Score:  0.9828178694158075 Time:  0.3813464641571045
   Youdens Index
0       1.000000
1       0.991770
2       0.977728
3       0.982372
4       1.000000
5       0.997701 

Alpha:  0.01
Score:  0.9673539518900344 Time:  0.4353961944580078
   Youdens Index
0       0.898777
1       0.968750
2       0.987829
3       0.982372
4       0.997959
5       0.993197 

Alpha:  0.1
Score:  0.9656357388316151 Time:  0.35732603073120117
   Youdens Index
0       0.946667
1       0.995885
2       0.953385
3       0.964744
4       1.000000
5       0.990898 

Alpha:  1


c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


Score:  0.9828178694158075 Time:  3.417895555496216
   Youdens Index
0       0.973333
1       0.987526
2       0.997930
3       0.964744
4       1.000000
5       0.993103 

Blue Config Alpha Tests
Alpha:  0.0001
Score:  0.9776632302405498 Time:  0.28500962257385254
   Youdens Index
0       0.931361
1       0.995885
2       0.983688
3       0.996071
4       1.000000
5       1.000000 

Alpha:  0.001
Score:  0.9759450171821306 Time:  0.27710485458374023
   Youdens Index
0       0.980750
1       1.000000
2       0.977728
3       0.943241
4       0.997959
5       0.995402 

Alpha:  0.01
Score:  0.9725085910652921 Time:  0.3039891719818115
   Youdens Index
0       0.973333
1       0.975051
2       0.997930
3       0.964744
4       0.987090
5       0.995402 

Alpha:  0.1
Score:  0.9570446735395189 Time:  0.3343040943145752
   Youdens Index
0       0.946667
1       0.971065
2       0.961415
3       0.984337
4       1.000000
5       0.990898 

Alpha:  1
Score:  0.979381443298969 Time:  3.449635

c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [38]:
# Red
mlpc_red =  MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1)
mlpc_red.fit(X_red_train, Y_train)
vector_red = mlpc_red.predict(X_red_train)

# Green
mlpc_green = MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1)
mlpc_green.fit(X_green_train, Y_train)
vector_green = mlpc_green.predict(X_green_train)

# Blue
mlpc_blue = MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1)
mlpc_blue.fit(X_blue_train, Y_train)
vector_blue = mlpc_blue.predict(X_blue_train)

# Validation
vector_red_val = mlpc_red.predict(X_red_validation)
vector_green_val = mlpc_green.predict(X_green_validation)
vector_blue_val = mlpc_blue.predict(X_blue_validation)


print("Red Model Validation")
red_val_predict = mlpc_red.predict(X_red_validation)
print(classification_report(Y_validation, red_val_predict, zero_division=0))
print(get_youdens_index(red_val_predict, Y_validation), "\n")

print("Green Model Validation")
green_val_predict = mlpc_green.predict(X_green_validation)
print(classification_report(Y_validation, green_val_predict, zero_division=0))
print(get_youdens_index(green_val_predict, Y_validation), "\n")

print("Blue Model Validation")
blue_val_predict = mlpc_blue.predict(X_blue_validation)
print(classification_report(Y_validation, blue_val_predict, zero_division=0))
print(get_youdens_index(blue_val_predict, Y_validation), "\n")

Red Model Validation
              precision    recall  f1-score   support

           0       0.99      0.97      0.98        75
           1       0.96      1.00      0.98        96
           2       1.00      0.98      0.99        99
           3       1.00      0.99      0.99        73
           4       1.00      1.00      1.00        92
           5       0.96      0.99      0.98       147

   micro avg       0.98      0.99      0.99       582
   macro avg       0.98      0.99      0.99       582
weighted avg       0.98      0.99      0.99       582
 samples avg       0.98      0.99      0.98       582

   Youdens Index
0       0.971361
1       0.991770
2       0.979798
3       0.986301
4       1.000000
5       0.979404 

Green Model Validation
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        75
           1       1.00      0.99      0.99        96
           2       0.96      0.97      0.96        99
           3       0.

In [45]:
# Stack the rgb predictions to get combi model values 
X_combined_train = np.column_stack((vector_red, vector_green, vector_blue)) # shape (1745, 18) - 18 features
X_combined_val = np.column_stack((vector_red_val, vector_green_val, vector_blue_val)) # shape (582, 18) - 18 features

# Combi Config HL Test
print("Combi Config HL Tests")
for hl in [(18), (14), (14, 7)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl, max_iter=1000)
    t0 = time.time()
    mlp.fit(X_combined_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_combined_val, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_combined_val), Y_validation), "\n")


print("Combi Config Alpha Tests")
# Combi Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=108, alpha=a, max_iter=300)
    t0 = time.time()
    mlp.fit(X_combined_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_combined_val, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_combined_val), Y_validation), "\n")

Combi Config HL Tests
Hidden layers:  18
Score:  0.9982817869415808 Time:  0.47547411918640137
   Youdens Index
0       1.000000
1       1.000000
2       1.000000
3       0.986301
4       1.000000
5       1.000000 

Hidden layers:  14
Score:  0.9965635738831615 Time:  0.5238513946533203
   Youdens Index
0       1.000000
1       1.000000
2       0.979798
3       1.000000
4       1.000000
5       1.000000 

Hidden layers:  (14, 7)
Score:  0.9621993127147767 Time:  0.710254430770874
   Youdens Index
0       0.946667
1       0.927083
2       0.967627
3       0.931507
4       0.978261
5       1.000000 

Combi Config Alpha Tests
Alpha:  0.0001
Score:  1.0 Time:  0.5675153732299805
   Youdens Index
0            1.0
1            1.0
2            1.0
3            1.0
4            1.0
5            1.0 

Alpha:  0.001
Score:  0.9982817869415808 Time:  0.5725200176239014
   Youdens Index
0       1.000000
1       1.000000
2       1.000000
3       0.986301
4       1.000000
5       1.000000 

Alpha: 

In [51]:
# Combi Train
mlpc_combi =  MLPClassifier(hidden_layer_sizes=(18), activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_combi.fit(X_combined_train, Y_train)

# Validation
print("Combined Model Validation")
combi_val = mlpc_combi.predict(X_combined_val)
print(classification_report(Y_validation, combi_val, zero_division=0))
print(get_youdens_index(combi_val, Y_validation), "\n")

Combined Model Validation
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        75
           1       1.00      1.00      1.00        96
           2       1.00      0.99      0.99        99
           3       1.00      0.99      0.99        73
           4       1.00      1.00      1.00        92
           5       1.00      1.00      1.00       147

   micro avg       1.00      1.00      1.00       582
   macro avg       1.00      1.00      1.00       582
weighted avg       1.00      1.00      1.00       582
 samples avg       1.00      1.00      1.00       582

   Youdens Index
0       1.000000
1       1.000000
2       0.989899
3       0.986301
4       1.000000
5       1.000000 



In [54]:
full_data_path = fr'../Datasets/full/Training'
X_red_train_final, X_red_validation_final, X_red_test_final, X_green_train_final, X_green_validation_final, X_green_test_final, X_blue_train_final, X_blue_validation_final, X_blue_test_final, Y_train_final, Y_validation_final, Y_test_final = process_files(full_data_path)

Starting...
Finished 



In [55]:
# Red Train Final
mlpc_red_final =  MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_red_final.fit(X_red_train_final, Y_train_final)
vector_red_final = mlpc_red_final.predict(X_red_train_final)

# Green Final
mlpc_green_final = MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_green_final.fit(X_green_train_final, Y_train_final)
vector_green_final = mlpc_green_final.predict(X_green_train_final)

# Blue Final
mlpc_blue_final = MLPClassifier(hidden_layer_sizes=(108), activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_blue_final.fit(X_blue_train_final, Y_train_final)
vector_blue_final = mlpc_blue_final.predict(X_blue_train_final)

# Validation Final
vector_red_val_final = mlpc_red_final.predict(X_red_validation_final)
vector_green_val_final = mlpc_green_final.predict(X_green_validation_final)
vector_blue_val_final = mlpc_blue_final.predict(X_blue_validation_final)


print("Red Model Validation Final")
red_val_predict_final = mlpc_red_final.predict(X_red_validation_final)
print(classification_report(Y_validation_final, red_val_predict_final, zero_division=0))
print(get_youdens_index(red_val_predict_final, Y_validation_final), "\n")

print("Green Model Validation Final")
green_val_predict_final = mlpc_green_final.predict(X_green_validation_final)
print(classification_report(Y_validation_final, green_val_predict_final, zero_division=0))
print(get_youdens_index(green_val_predict_final, Y_validation_final), "\n")

print("Blue Model Validation Final")
blue_val_predict_final = mlpc_blue_final.predict(X_blue_validation_final)
print(classification_report(Y_validation_final, blue_val_predict_final, zero_division=0))
print(get_youdens_index(blue_val_predict_final, Y_validation_final), "\n")

Red Model Validation Final
              precision    recall  f1-score   support

           0       0.76      0.28      0.41       131
           1       0.84      0.87      0.85       119
           2       0.00      0.00      0.00       128
           3       0.62      0.37      0.47       131
           4       0.00      0.00      0.00       129
           5       0.94      0.38      0.54       131
           6       0.79      0.91      0.84       122
           7       0.67      0.14      0.23       131
           8       0.00      0.00      0.00       131
           9       0.63      0.50      0.55       115
          10       0.71      0.37      0.49       131
          11       0.58      0.39      0.47       131
          12       0.82      0.56      0.66       178
          13       0.00      0.00      0.00       131
          14       0.92      0.89      0.90       114
          15       0.78      0.82      0.80       131
          16       0.87      0.73      0.80       131


              precision    recall  f1-score   support

           0       0.95      0.99      0.97       131
           1       0.97      0.97      0.97       119
           2       0.87      0.10      0.18       128
           3       0.88      0.69      0.77       131
           4       0.82      0.88      0.85       129
           5       0.78      1.00      0.88       131
           6       0.91      0.84      0.88       122
           7       0.93      0.76      0.83       131
           8       0.75      0.91      0.82       131
           9       0.96      0.69      0.80       115
          10       0.96      0.90      0.93       131
          11       0.92      0.91      0.92       131
          12       0.93      0.90      0.91       178
          13       0.82      0.81      0.82       131
          14       0.92      0.93      0.93       114
          15       0.93      0.68      0.78       131
          16       0.93      0.81      0.87       131
          17       0.88    

     Youdens Index
0         0.976875
1         0.999944
2         0.881747
3         0.846711
4         0.999215
5         0.976987
6         0.934034
7         0.953076
8         0.969073
9         0.886844
10        0.694152
11        0.892513
12        0.943820
13        0.999832
14        0.999888
15        0.785699
16        0.915750
17        0.917131
18        0.885216
19        0.805025
20        0.999383
21        0.808992
22        0.979908
23        0.967951
24        0.930793
25        0.962060
26        0.977099
27        0.968642
28        0.809761
29        0.984733
30        1.000000
31        0.992198
32        0.851128
33        0.984340
34        0.534071
35        0.808277
36        0.917746
37        0.818655
38        0.992000
39        0.816626
40        0.935888
41        0.906333
42        0.531310
43        0.890998
44        0.934966
45        0.885103
46        0.957568
47        0.997980
48        0.953862
49        0.967917
50        0.992310
51        0.

In [56]:
# Stack the final rgb predictions to get final combi model values 
X_combined_train_final = np.column_stack((vector_red_final, vector_green_final, vector_blue_final)) # shape (1745, 18) - 18 features
X_combined_val_final = np.column_stack((vector_red_val_final, vector_green_val_final, vector_blue_val_final)) # shape (582, 18) - 18 features

# Combi Train Final
mlpc_combi_final =  MLPClassifier(hidden_layer_sizes=(18), activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_combi_final.fit(X_combined_train_final, Y_train_final)

# Validation Final
print("Combined Model Validation")
combi_val_final = mlpc_combi_final.predict(X_combined_val_final)
print(classification_report(Y_validation_final, combi_val_final, zero_division=0))
print(get_youdens_index(combi_val_final, Y_validation_final), "\n")

Combined Model Validation
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       131
           1       1.00      0.99      1.00       119
           2       0.93      0.88      0.91       128
           3       0.97      0.89      0.93       131
           4       0.98      0.97      0.98       129
           5       0.98      0.96      0.97       131
           6       1.00      0.97      0.98       122
           7       0.97      0.96      0.97       131
           8       0.99      0.97      0.98       131
           9       1.00      0.92      0.96       115
          10       1.00      0.93      0.96       131
          11       0.98      0.97      0.98       131
          12       0.99      0.97      0.98       178
          13       1.00      1.00      1.00       131
          14       0.99      0.97      0.98       114
          15       0.98      0.91      0.94       131
          16       0.99      0.90      0.94       131
 